In [1]:
from extract_ner import get_all_ners,get_all_projects
from pdf_analyzer.config_load import load_config
from pdf_analyzer.api import API
from omegaconf import OmegaConf

In [2]:
from transformers import pipeline

pipe = pipeline("token-classification", model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
server_config = load_config("config/api/grobid-server-config.yaml")
extract_config = load_config("config/api/api-base-config.yaml")
print("SERVER_CONFIG\n"+OmegaConf.to_yaml(server_config))
print("CLOUD_CONFIG\n"+OmegaConf.to_yaml(extract_config))

base_api = API.BaseAPI(extract_config,server_config)

files = base_api.proccesed_files

SERVER_CONFIG
url:
  protocol: http
  api_domain: yordi111nas.synology.me
  port: 8070

CLOUD_CONFIG
data:
  data_dir: data/PDFs
  format: .pdf
  recursive: true
grobid:
  cache: true
  cache_dir: data/xmls
  operation_key: processFulltextDocument
  format: .grobid.tei.xml
  recursive: true

http://yordi111nas.synology.me:8070/api/isalive
GROBID server is up and running
data/xmls\nlp\Bert.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\DistillBERT.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\Dont_stop_pretraining.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\GPT-3.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LIME.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LoRA.grobid.tei.xml already exist, skipping... (use --force to repro

In [4]:
all_orgs_rel,all_orgs=get_all_ners(files,pipe)
all_orgs_rel


[{'name': 'Al-lenNLP',
  'score': 0.67498153,
  'type': 'ORG',
  'org_id': 0,
  'paper_id': 2},
 {'name': 'Office of Naval Research',
  'score': 0.9967265,
  'type': 'ORG',
  'org_id': 1,
  'paper_id': 2},
 {'name': 'Google',
  'score': 0.99887425,
  'type': 'ORG',
  'org_id': 2,
  'paper_id': 2},
 {'name': 'OpenAI',
  'score': 0.9626693,
  'type': 'ORG',
  'org_id': 3,
  'paper_id': 3},
 {'name': 'WebText',
  'score': 0.7112076,
  'type': 'ORG',
  'org_id': 4,
  'paper_id': 3},
 {'name': 'TerraSwarm',
  'score': 0.99291795,
  'type': 'ORG',
  'org_id': 5,
  'paper_id': 4},
 {'name': 'STARnet',
  'score': 0.87451035,
  'type': 'ORG',
  'org_id': 6,
  'paper_id': 4},
 {'name': 'Semiconductor Research Corporation',
  'score': 0.9271019,
  'type': 'ORG',
  'org_id': 7,
  'paper_id': 4},
 {'name': 'MARCO',
  'score': 0.9814992,
  'type': 'ORG',
  'org_id': 8,
  'paper_id': 4},
 {'name': 'DARPA',
  'score': 0.976103,
  'type': 'ORG',
  'org_id': 9,
  'paper_id': 4}]

In [5]:
all_orgs

[{'name': 'Al-lenNLP', 'type': 'ORG', 'org_id': 0},
 {'name': 'Office of Naval Research', 'type': 'ORG', 'org_id': 1},
 {'name': 'Google', 'type': 'ORG', 'org_id': 2},
 {'name': 'OpenAI', 'type': 'ORG', 'org_id': 3},
 {'name': 'WebText', 'type': 'ORG', 'org_id': 4},
 {'name': 'TerraSwarm', 'type': 'ORG', 'org_id': 5},
 {'name': 'STARnet', 'type': 'ORG', 'org_id': 6},
 {'name': 'Semiconductor Research Corporation', 'type': 'ORG', 'org_id': 7},
 {'name': 'MARCO', 'type': 'ORG', 'org_id': 8},
 {'name': 'DARPA', 'type': 'ORG', 'org_id': 9}]

In [6]:
all_projects,all_projects_relation=get_all_projects(files)
all_projects

[{'project_name': 'MURI',
  'project_federal_id': 'N00014-18-1-2670',
  'project_id': 0},
 {'project_name': 'ONR',
  'project_federal_id': '#N00014-13-1-0023',
  'project_id': 1}]

In [14]:
all_projects_relation

[{'project_name': 'MURI',
  'project_federal_id': 'N00014-18-1-2670',
  'project_id': 0,
  'paper_id': 2},
 {'project_name': 'ONR',
  'project_federal_id': '#N00014-13-1-0023',
  'project_id': 1,
  'paper_id': 4},
 {'project_name': 'ONR',
  'project_federal_id': '#W911NF-13-1-0246',
  'project_id': 1,
  'paper_id': 4}]

In [8]:
import re
def extract_award_identifiers(text):
    # Expresiones regulares para identificadores de premios
    regex_patterns = {
        "NIH": r'(?:#)?\b[1-9][A-Z\d]{3}[A-Z]{2}\d{6}(?:-[AS]?\d+)?\b',
        "DOD": r'(?:#)?\b[A-Z\d]{6}-\d{2}-[123]-\d{4}\b',
        "NASA": r'(?:#)?\b(?:80|NN)[A-Z]+\d{2}[A-Z\d]+\b',
        "Education": r'(?:#)?\b[A-Z]+\d+[A-Z]\d{2}[A-Z\d]+\b'
    }

    # Lista para almacenar todos los identificadores únicos encontrados
    all_identifiers = []

    # Aplicar cada expresión regular al texto y almacenar los resultados
    for pattern in regex_patterns.values():
        identifiers = re.findall(pattern, text)
        all_identifiers.extend(identifiers)

    # Eliminar duplicados
    unique_identifiers = list(set(all_identifiers))

    return unique_identifiers

In [9]:


texto_ejemplo = """
Aquí hay algunos identificadores de premios:

NIH: 1R01CA123456-01A1, #2R01CA654321-03B2
DOD: W81XWH-22-1-1234, #W912D1-21-3-9012
NASA: NNC21AA01H, #GRC20G-1234, NNC21AA02A
Department of Education: P031B201234, #T042A190567
"""

# Extraer identificadores de premios del texto de ejemplo
identificadores = extract_award_identifiers(texto_ejemplo)

# Imprimir los identificadores encontrados para cada agencia
for ids in identificadores:
    print(f"Identificadores de premios {ids}")

Identificadores de premios #2R01CA654321
Identificadores de premios P031B201234
Identificadores de premios #W912D1-21-3-9012
Identificadores de premios W81XWH-22-1-1234
Identificadores de premios #T042A190567
Identificadores de premios 1R01CA123456
Identificadores de premios NNC21AA02A
Identificadores de premios NNC21AA01H


In [10]:
def combine_identifiers_with_names(text):
    # Obtener identificadores de premios del texto
    identifiers = extract_award_identifiers(text)
    
    # Lista para almacenar nombres y códigos asociados
    names_and_codes = []

    # Iterar sobre los identificadores y buscar los nombres correspondientes
    for identifier in identifiers:
        # Buscar el nombre del proyecto antes del identificador
        match = re.search(rf'(\w+)\s+(?:grant|award|code)\s+{re.escape(identifier)}', text, re.IGNORECASE)
        if match:
            names_and_codes.append((match.group(1), identifier))

    return names_and_codes

In [11]:
def get_projects_names(text):
    # Obtener identificadores de premios del texto
    identifiers = extract_award_identifiers(text)
    
    # Lista para almacenar nombres y códigos asociados
    names_and_codes = []

    # Iterar sobre los identificadores y buscar los nombres correspondientes
    for identifier in identifiers:
        # Buscar el nombre del proyecto antes del identificador
        match = re.search(rf'(\w+)\s+(?:grant|award|code)\s+{re.escape(identifier)}', text, re.IGNORECASE)
        if match:
            names_and_codes.append(match.group(1))

    return names_and_codes

In [12]:
texto_ejemplo = """
Aquí hay algunos identificadores de premios con nombres asociados:

MURI grant N00014-18-1-2670, MURI award N00014-18-1-2670
DOD Award W81XWH-18-1-1234, Cooperative Agreement W911NF-20-2-5678
NASA Grant NNC21AA01H, Research Cooperative Agreement GRC20G-1234, NNC21AA02A
Department of Education Grant P031B201234, Grant T042A190567
"""

# Extraer nombres asociados con códigos del texto de ejemplo
nombres_con_codigos = get_projects_names(texto_ejemplo)
nombres_con_codigos

['DOD', 'Education', 'NASA', 'MURI']

In [13]:
nombres_con_codigos

['DOD', 'Education', 'NASA', 'MURI']